In [52]:
# training model
import warnings
import pandas as pd
from sklearn.linear_model import LinearRegression, ElasticNet, Lasso, Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split, KFold, cross_val_score

In [53]:
filename = "data_excluded/forestfires.csv"

names = [
    "X",
    "Y",
    "month",
    "day",
    "FFMC",
    "DMC",
    "DC",
    "ISI",
    "temp",
    "RH",
    "wind",
    "rain",
    "area",
]

df = pd.read_csv(filename, names=names)

In [54]:
# convert month and day to categorical
df.month.replace({
    "jan": 1,
    "feb": 2,
    "mar": 3,   
    "apr": 4,
    "may": 5,
    "jun": 6,
    "jul": 7,
    "aug": 8,   
    "sep": 9,
    "oct": 10,
    "nov": 11,
    "dec": 12
}, inplace=True)

df.day.replace({
    "mon": 1,
    "tue": 2,
    "wed": 3,
    "thu": 4,
    "fri": 5,
    "sat": 6,
    "sun": 7
}, inplace=True)

df.head()

C:\Users\traya\AppData\Local\Temp\ipykernel_25472\299477150.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df.month.replace({
C:\Users\traya\AppData\Local\Temp\ipykernel_25472\299477150.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.month.replace({
C:\Users\traya\AppData\Local\Temp\ipyk

,X,Y,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area
0,7,5,3,5,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.0
1,7,4,10,2,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.0
2,7,4,10,6,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.0
3,8,6,3,5,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.0
4,8,6,3,7,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.0


In [55]:
array = df.values
X = array[:, 0:12]  # first 12 columns from the dataset
Y = array[:, 12]  # 13th column is the target label

In [56]:
# scoring metrics
max_error_scoring = 'neg_max_error'
neg_mean_absolute_error_scoring = 'neg_mean_absolute_error'
r2_scoring = 'r2'
neg_mean_squared_error_scoring = 'neg_mean_squared_error'

In [57]:
models = []
models.append(('Linear Regression', LinearRegression()))
models.append(('Elastic Net', ElasticNet()))
models.append(('Lasso', Lasso()))
models.append(('Ridge', Ridge()))
models.append(('K-Neighbors', KNeighborsRegressor()))
models.append(('Support Vector Machine', SVR()))
models.append(('Decision Tree', DecisionTreeRegressor()))

In [58]:
import pandas as pd

# Evaluate models and print results
results = []
names = []
for name, model in models:
    kfold = KFold(n_splits=10, random_state=7, shuffle=True)
    cv_results = cross_val_score(model, X, Y, cv=kfold, scoring=max_error_scoring)
    cv_results2 = cross_val_score(model, X, Y, cv=kfold, scoring=neg_mean_absolute_error_scoring)
    cv_results3 = cross_val_score(model, X, Y, cv=kfold, scoring=r2_scoring)
    cv_results4 = cross_val_score(model, X, Y, cv=kfold, scoring=neg_mean_squared_error_scoring)
    results.append([
        name,
        cv_results.mean(),
        -cv_results2.mean(),
        cv_results3.mean(),
        -cv_results4.mean()
    ])

# Print results as a table
results_df = pd.DataFrame(
        results,
        columns=["Model", "Max_Error", "Mean_Absolute_Error", "R2", "Mean_Squared_Error"]
    )
print(results_df.to_string(index=False))

                 Model   Max_Error  Mean_Absolute_Error         R2  Mean_Squared_Error
     Linear Regression -278.651811            19.923961  -0.448206         4072.408893
           Elastic Net -273.461816            19.429581  -0.283493         4042.781243
                 Lasso -273.551076            19.453062  -0.290731         4044.489363
                 Ridge -276.361753            19.877511  -0.398110         4064.191522
           K-Neighbors -313.945400            19.635797  -1.501334         4475.485429
Support Vector Machine -293.549242            12.844258  -0.118160         4190.566565
         Decision Tree -500.777000            23.524026 -10.169194         7156.824528


In [59]:
# actual training

# split the dataset into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=1, shuffle=True)

In [60]:
# training with Lasso regression
lasso_model = Lasso()
lasso_model.fit(X_train, Y_train)

,alpha,1.0
,fit_intercept,True
,precompute,False
,copy_X,True
,max_iter,1000
,tol,0.0001
,warm_start,False
,positive,False
,random_state,None
,selection,'cyclic'


In [61]:
predictions = lasso_model.predict(X_test)
print(predictions)


[ 2.94995557 10.31741163  5.11331167  8.82953603  9.37472473  2.25890375
 16.70956165 10.21372406 12.71398315 17.82743566 12.00661147 -0.13366089
  2.05100125  9.61362391 20.35978817  4.46999326 11.4878202   8.22013444
 22.99636305  6.57213204 15.16990813  2.06185639  2.60841973 20.26525161
 17.97317314 15.84725657 11.22294233 18.07415259 19.27700019  6.71476154
 11.69520281  7.95309194 12.58956125  7.84373939 12.95490693  7.86654371
 10.29230298 13.12601655  7.55670958 18.24618339 10.86374712  0.84185952
 20.22670534  8.67143132 17.11750646  9.87835433 -0.41212642 12.75284261
  7.16986389  4.50527181 26.3280149  21.87386519 20.89573415 16.78773238
 21.87773448 16.31145194 12.17760756 21.10444723 19.24150053 13.21816328
  8.7358127   2.11532756 23.43585742  8.45085449 13.32288157  1.14739124
  7.81763591 13.16758569 12.31864714  7.13142564 21.82793694 15.85504258
  7.50956744  9.03027599  9.14037809 10.44621761 19.90381093 22.18118018
 10.29641445 15.17851613  8.17927973 16.67198374  9

In [62]:
# deploying model

import pickle

pickle.dump(lasso_model, open("data_excluded/lasso_model.pkl", "wb"))

model = pickle.load(open("data_excluded/lasso_model.pkl", "rb"))

In [63]:
# flask and ngroc API

from flask import Flask, request
from flask_ngrok import run_with_ngrok
import requests
import numpy as np
app = Flask(__name__)
run_with_ngrok(app)   #starts ngrok when the app is run


In [64]:
# run server

@app.route('/predict',methods=['POST'])
def home():
    X = int(request.args.get('X', ''))
    Y = int(request.args.get('Y', ''))
    month = int(request.args.get('month', ''))
    day = int(request.args.get('day', ''))
    FFMC = float(request.args.get('FFMC', ''))
    DMC = float(request.args.get('DMC', ''))
    DC = float(request.args.get('DC', ''))
    ISI = float(request.args.get('ISI', ''))
    temp = float(request.args.get('temp', ''))
    RH = float(request.args.get('RH', ''))
    wind = float(request.args.get('wind', ''))
    rain = float(request.args.get('rain', ''))
    prediction = lasso_model.predict([[X, Y, month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain]])
    print('*******************************')
    print(prediction)
    return 'Prediction is ' + str(prediction[0])
      
# app.run()